- [ ] Setup some kernels and check perf for sparse matrices
- [ ] plug in custom vjp for scipy sparse, ispc spmv & cuda spmv

In [1]:
import scipy.io
import os.path
import numpy as np

In [2]:
mats = scipy.io.loadmat(os.path.expanduser('~/Downloads/matrices.mat'))
SC = mats['SC']
LC = mats['LC']

FileNotFoundError: [Errno 2] No such file or directory: '/Users/duke/Downloads/matrices.mat'

In [45]:
nn, _ = LC.shape

In [46]:
x = np.zeros((nn, ), 'f') + 0.5

In [11]:
%timeit SC @ x
%timeit LC @ x

7.04 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.86 ms ± 47.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
import jax.experimental.sparse as jsp
import jax
import jax.numpy as jnp

In [32]:
SCcoo = SC.tocoo()

In [37]:
jSC = jsp.BCOO((jnp.array(SCcoo.data), jnp.c_[jnp.array(SCcoo.row), jnp.array(SCcoo.col)]), shape=jSC.shape)

In [39]:
f = jax.jit(lambda x: jSC @ x)
%timeit f(jx)

2023-01-18 23:06:12.326336: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  gather.19 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2023-01-18 23:06:13.089839: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 1.763769362s
Constant folding an instruction is taking > 1s:

  gather.19 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't n

35.8 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


So the Jax native thing is 5x slower.  That's not great for single sims but probably absorbable for any inference related thing.

## Plug Scipy's csr into Jax

Let's plug scipy's csr into Jax, assuming symmetric sparse matrix for which we don't need the gradient.

intermediate issues:

- scipy csr_matrix isn't a type Jax understands
- and it assumes numpy arrays, not Jax arrays

*TODO* finish this up.

In [48]:
import collections
import scipy.sparse

CSR = collections.namedtuple('data indices indptr shape')

@jax.custom_vjp
def symm_spmv(csr: CSR, x):
    A = scipy.sparse.csr_matrix((csr.data, csr.indices, csr.indptr), shape=csr.shape)
    return A @ x

def symm_spmv_fwd(csr: CSR, x):
    A = scipy.sparse.csr_matrix((csr.data, csr.indices, csr.indptr), shape=csr.shape)
    return symm_spmv(A, x), (A, )

def symm_spmv_bwd(res, g):
    A, = res
    return None, symm_spmv(A, g)

symm_spmv.defvjp(symm_spmv_fwd, symm_spmv_bwd)

symm_spmv(SC, x)

TypeError: Value <81938x81938 sparse matrix of type '<class 'numpy.float32'>'
	with 9358471 stored elements in Compressed Sparse Column format> with type <class 'scipy.sparse._csc.csc_matrix'> is not a valid JAX type